In [1]:
import pandas as pd

from IPython.display import HTML

In [2]:
dh = pd.read_csv('sequencias.csv')

In [3]:
dh

,demanda,sequencia
0,38,"(('DESCONHECIDO', 'Estimar pontos de função'),..."
1,73,"(('DESCONHECIDO', 'Estimar pontos de função'),..."
2,74,"(('DESCONHECIDO', 'Estimar pontos de função'),..."
3,77,"(('Realizar implementação', 'Elaborar procedim..."
4,78,"(('Realizar implementação', 'Elaborar procedim..."
...,...,...
23180,23446,"(('Novo', 'Novo'))"
23181,23447,"(('Novo', 'Desenvolver internamente'), ('Desen..."
23182,23448,"(('Novo', 'Novo'))"
23183,23449,"(('Novo', 'Novo'))"


In [9]:
dh.loc[dh.demanda==38, 'sequencia'].values[0]

"(('DESCONHECIDO', 'Estimar pontos de função'), ('Estimar pontos de função', 'Elaborar parecer: desenvolver ou adquirir?'), ('Elaborar parecer: desenvolver ou adquirir?', 'Considerar decisão da DISIS: ratificar ou retificar'), ('Considerar decisão da DISIS: ratificar ou retificar', 'Atualizar registro da demanda'), ('Atualizar registro da demanda', 'Demanda sem ônus em pontos de função concluída'))"

In [5]:
dh.nunique()

demanda      23185
sequencia      581
dtype: int64

In [6]:
HTML(dh.groupby(['sequencia']).nunique().sort_values(by=['demanda'], ascending=False).to_html())

,demanda,sequencia
sequencia,,
"(('Novo', 'Em andamento'), ('Em andamento', 'Concluído'))",11844,1
"(('Novo', 'Desenvolver internamente'), ('Desenvolver internamente', 'Demanda sem ônus em pontos de função concluída'))",1824,1
"(('Novo', 'Demanda sem ônus em pontos de função concluída'))",1445,1
"(('Novo', 'Preparar documentação para fábrica'), ('Preparar documentação para fábrica', 'Pronto para fábrica'), ('Pronto para fábrica', 'Realizar implementação'), ('Realizar implementação', 'Testar implementação'), ('Testar implementação', 'Homologar com Demandante'), ('Homologar com Demandante', 'Entregar artefatos'), ('Entregar artefatos', 'Validar artefatos'), ('Validar artefatos', 'Contar pontos de função'), ('Contar pontos de função', 'Atualizar registro da demanda'), ('Atualizar registro da demanda', 'Demanda com ônus em pontos de função concluída'))",1190,1
"(('Novo', 'Em andamento'), ('Em andamento', 'Concluído'), ('Concluído', 'Aceito'))",1006,1
"(('Novo', 'Desenvolver internamente'), ('Desenvolver internamente', 'Validar com a Área Demandante'), ('Validar com a Área Demandante', 'Demanda sem ônus em pontos de função concluída'))",534,1
"(('Novo', 'Demanda cancelada'))",495,1
"(('Preparar documentação para fábrica', 'Atualizar registro da demanda'), ('Atualizar registro da demanda', 'Demanda sem ônus em pontos de função concluída'))",451,1
"(('Novo', '(SISCAP) Em Análise'), ('(SISCAP) Em Análise', 'Desenvolver internamente'), ('Desenvolver internamente', 'Demanda sem ônus em pontos de função concluída'))",327,1
